# Are statin prescribing ratios different for dispensing practices?

#### 1. Obtain GP Prescribing data
#### 2. Cut GP Prescribing data for things of interest (statins)
#### 3. Obtain list of dispensing practices
#### 4. Combine statin cut and dispensing practice list
#### 5. Compare dispensing vs non-dispensing statin prescribing rations

In [57]:
import requests
from StringIO import StringIO
from numpy import nan
import pandas as pd
import xlrd 
import csv

In [ ]:
url = 'http://datagov.ic.nhs.uk/presentation/2015_01_January/T201501PDPI+BNFT.CSV' #gp prescribing data (Jan 2015)
r = requests.get(url)
data = r.content
df = pd.read_csv(StringIO(data))


In [ ]:
df.columns = [x.strip() for x in df.columns] #gets rid of variable whitespace

In [ ]:
df = df[df['BNF NAME'].str.contains('statin')] #cut for rows with statin in them and throw away unwanted statins
df = df[~df['BNF NAME'].str.contains('Nystatin')]
df = df[~df['BNF NAME'].str.contains('Sandostatin')]
df = df[~df['BNF NAME'].str.contains('Ecostatin')]

In [ ]:
df.head()

In [ ]:
df[df['BNF NAME'].isin(statins)]

In [1]:
ls

datas/  LICENSE  pa-research.ipynb  README.md


In [2]:
cd datas/

/home/sam/Dropbox/Documents/pa-research/datas


In [3]:
ls

Disp Pracs Name and Address 2015-01-31.xls


In [38]:
excel_file = 'Disp Pracs Name and Address 2015-01-31.xls'

In [99]:
df1 = pd.read_excel(excel_file)

In [40]:
df1.columns

Index([u'Dispensing Practices Address Details', u'Unnamed: 1', u'Unnamed: 2', u'Unnamed: 3', u'Unnamed: 4', u'Unnamed: 5', u'Unnamed: 6'], dtype='object')

In [102]:
df1['Dispensing Practices Address Details'].tail()

NaN                                          Report For:
NaN                            Practice Name and Address
NaN    Aylmer Lodge Cookley Partnership, Cookley Surg...
NaN    Chaddesley Surgery, The Surgery, Hemmingway, C...
NaN    Wolverley Surgery, The Surgery, Wolverley, Kid...
Name: Dispensing Practices Address Details, dtype: object

In [42]:
len(df1)

1562

In [97]:
df1['Dispensing Practices Address Details'] = df1['Dispensing Practices Address Details'].str.replace('Primary Care Trust:', 'NaN')
df1['Dispensing Practices Address Details'] = df1['Dispensing Practices Address Details'].str.replace('Report For:', 'NaN')
df1['Dispensing Practices Address Details'] = df1['Dispensing Practices Address Details'].str.replace('Primary Care Trust:', 'NaN')


In [98]:
df1['Dispensing Practices Address Details'].dropna()

Series([], name: Dispensing Practices Address Details, dtype: float64)

In [65]:
df1['Dispensing Practices Address Details'].unique()

array([ nan])